In [14]:
#3.5.7
# Load the TensorBoard notebook extension
#%load_ext tensorboard
import tensorflow as tf
import datetime
from keras.datasets import reuters

# Clear any logs from previous runs
!rm -rf ./logs/ 

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words=10000)

print(len(train_data))
print(len(test_data))

word_index = reuters.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
decoded_newswire = ' '.join([reverse_word_index.get(i - 3, '?') for i in train_data[0]])

import numpy as np

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

def to_one_hot(labels, dimension=46):
    results = np.zeros((len(labels), dimension))
    for i, label in enumerate(labels):
        results[i, label] = 1
    return results

one_hot_train_labels = to_one_hot(train_labels)
one_hot_test_labels = to_one_hot(test_labels)

from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(46, activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics='accuracy')

#3.5.4
x_val = x_train[:1000]
partial_x_train = x_train[1000:]

y_val = one_hot_train_labels[:1000]
partial_y_train = one_hot_train_labels[1000:]

#3.18
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=4,
                    batch_size=1024,
                    validation_data=(x_val, y_val),
                    callbacks=[tensorboard_callback])

results = model.evaluate(x_test, one_hot_test_labels)


2021-11-10 18:06:36.232899: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-11-10 18:06:36.232929: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-11-10 18:06:36.234451: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.


8982
2246
Epoch 1/4
5/8 [=================>............] - ETA: 0s - loss: 3.3499 - accuracy: 0.3877

2021-11-10 18:06:38.385482: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-11-10 18:06:38.385498: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-11-10 18:06:38.449051: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-11-10 18:06:38.450380: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-11-10 18:06:38.452174: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/fit/20211110-180636/train/plugins/profile/2021_11_10_18_06_38

2021-11-10 18:06:38.453425: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to logs/fit/20211110-180636/train/plugins/profile/2021_11_10_18_06_38/Roys-MacBook-Pro.local.trace.json.gz
2021-11-10 18:06:38.455935: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/fit/20211110-180636/tr

8/8 [==============================] - 1s 65ms/step - loss: 3.0306 - accuracy: 0.4493 - val_loss: 2.1629 - val_accuracy: 0.5990
Epoch 2/4
8/8 [==============================] - 0s 32ms/step - loss: 1.7992 - accuracy: 0.6626 - val_loss: 1.6025 - val_accuracy: 0.6550
Epoch 3/4
8/8 [==============================] - 0s 31ms/step - loss: 1.3457 - accuracy: 0.7162 - val_loss: 1.3453 - val_accuracy: 0.7110
Epoch 4/4
71/71 [==============================] - 0s 1ms/step - loss: 1.2826 - accuracy: 0.7084


In [15]:
#3.5.7
print(history.history['accuracy'][-1])
print(history.history['val_accuracy'][-1])

predictions = model.predict(x_test)
print(predictions[0])
print(np.sum(predictions[0]))
print(np.argmax(predictions[0]))

0.7662240266799927
0.7260000109672546
[3.4751758e-04 2.5782313e-03 3.1365579e-04 7.2386110e-01 2.2603354e-01
 1.0128936e-04 4.1682468e-04 3.6127807e-04 1.2020085e-02 1.0096133e-03
 1.8967845e-03 2.5873040e-03 4.5888644e-04 5.8523950e-04 7.1959023e-04
 4.4577668e-05 6.6329162e-03 4.1204513e-04 3.1307453e-04 4.2319736e-03
 4.8699686e-03 1.6138119e-03 1.2371814e-04 7.3148694e-04 6.0704513e-04
 6.3406880e-04 1.5895697e-04 1.6977811e-04 5.8693334e-04 1.7857563e-04
 6.2902825e-04 6.0561945e-04 6.7400069e-05 1.5404346e-04 4.0495946e-04
 2.0502580e-04 5.7586958e-04 4.8262045e-05 4.1421523e-04 2.6869628e-04
 4.0581406e-04 7.6552178e-04 3.2325758e-04 9.0902373e-05 1.0381099e-04
 3.3768031e-04]
0.9999999
3
